### 03INFER.ipynb

the Infer class using LLE-like method

06/13/2020

In [ ]:
class InferSet(Dataset):
    def __init__(self,df):
        super().__init__()
        self.df = df
        self.qids = df['query_id'].unique().tolist()
    def __len__(self):
        return len(self.qids)
    def get_row_features(self, rows):
        rois = [x['features'] for idx, x in rows.iterrows()]
        idxs = np.cumsum([0] + [x['features'].shape[0] for idx, x in rows.iterrows()])
        rois = np.vstack(rois)
         
        from sklearn.neighbors import NearestNeighbors
        nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(rois)
        distances, indices = nbrs.kneighbors(rois)
        rois = 0.7 * rois + 0.2 * rois[[indices[:,1]]] + \
                            0.1 * rois[[indices[:,2]]]
        return [rois[idxs[i]:idxs[i+1], :] for i in range(len(rows))]
    
    def __getitem__(self, idx):
        qid = self.qids[idx]
        rows = self.df[self.df['query_id']==qid].reset_index()
        
        fisrt_row = rows.iloc[0]
        tokens, attn1 = pad_token(fisrt_row['query_tokenid'])
        
        rois_list = []
        attn2_list = []
        row_features = self.get_row_features(rows) # LLE-like method transformation
        
        for idx,row in rows.iterrows():
            rois, attn2 = pad_rois (row_features[idx])
            rois_list.append(rois[None,:])
            attn2_list.append(attn2[None,:])
        rois = np.vstack(rois_list)
        attn2 = np.vstack(attn2_list)
        return tokens, attn1, rois, attn2, fisrt_row['query_id'], rows['product_id'].values